# Analysis DATA

In [3]:
from utilities import *

raw_data = pd.read_excel("data/Datos_Market_copy.xlsx")

sa = SalesAnalysis(raw_data)

data = sa.data

/Users/oscarjimenezbou/Documents/TFG_ADE/code/utilities.py:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["brand"].replace({i: "other"}, inplace=True)
/Users/oscarjimenezbou/Documents/TFG_ADE/code/utilities.py:125: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(
/Users/oscarjimenezbou/Documents/TFG_ADE/code/utilities.py:125: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(
/Users/oscarjimenez

### Create our model

In [4]:
model_brand35 = sa.modelization(
    data[sa.brand35], sa.__interactions_delected_brand35__()
)
model_brand35.summary()

/Users/oscarjimenezbou/Documents/TFG_ADE/code/utilities.py:596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered_by_brand.rename(


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           volume_sales   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.330e+04
Date:                Sun, 03 Nov 2024   Prob (F-statistic):               0.00
Time:                        11:27:46   Log-Likelihood:                -6872.2
No. Observations:                 805   AIC:                         1.379e+04
Df Residuals:                     783   BIC:                         1.389e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
==============================================================================================================================
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------
Intercept                                                     97.5272    110.700      0.881      0.379    -119.778     314.832
supermarket_supermarket_C[T.True]                            840.8067    193.898      4.336      0.000     460.186    1221.427
variant_standard[T.True]                                    -625.6858    167.465     -3.736      0.000    -954.420    -296.952
pack_size_351___500_GR[T.True]                               567.7458    264.652      2.145      0.032      48.235    1087.257
supermarket_supermarket_C[T.True]:variant_standard[T.True]  1742.2915    263.573      6.610      0.000    1224.899    2259.684
pack_size_351___500_GR[T.True]:variant_standard[T.True]     1960.4504    333.648      5.876      0.000    1305.499    2615.402
unit_sales                                                     0.1532      0.008     18.384      0.000       0.137       0.170
unit_sales:supermarket_supermarket_B[T.True]                  -0.2273      0.006    -36.576      0.000      -0.239      -0.215
unit_sales:supermarket_supermarket_C[T.True]                  -0.0883      0.007    -12.698      0.000      -0.102      -0.075
unit_sales:supermarket_supermarket_D[T.True]                  -0.0362      0.001    -45.426      0.000      -0.038      -0.035
unit_sales:variant_light[T.True]                               0.0398      0.004     11.212      0.000       0.033       0.047
unit_sales:variant_vegan[T.True]                               0.1595      0.025      6.450      0.000       0.111       0.208
unit_sales:pack_size_351___500_GR[T.True]                      0.3248      0.009     37.021      0.000       0.308       0.342
unit_sales:pack_size_501___700_GR[T.True]                      0.5393      0.011     50.116      0.000       0.518       0.560
unit_sales:pack_size_701___1000_GR[T.True]                     0.7407      0.020     36.147      0.000       0.701       0.781
value_sales                                                    0.1255      0.009     14.727      0.000       0.109       0.142
value_sales:supermarket_supermarket_B[T.True]                  0.1666      0.006     26.700      0.000       0.154       0.179
value_sales:supermarket_supermarket_C[T.True]                  0.0305      0.007      4.556      0.000       0.017       0.044
value_sales:variant_light[T.True]                             -0.1484      0.009    -16.582      0.000      -0.166      -0.131
value_sales:variant_standard[T.True]                          -0.1069      0.009    -12.271      0.000      -0.124      -0.090
value_sales:variant_vegan[T.True]                             -0.1521      0.030     -5.017      0.000      -0.212      -0.093
value_sales:pack_size_701___1000_GR